# training 10 features to 4 classes
## one vs all

weights is size 10 (should be 11 ?)
replacing nan with 0

Removing


replacing nan with 0

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class LogRegTrain:
    def __init__(self, df_to_train, df_class):
        ''' standardize df to train'''
        df_std = df_to_train.agg(lambda feature: LogRegTrain.standardize(feature))
        self.df_std= df_std
        self.x_train = np.array(df_std.fillna(0))  
        self.features = df_std.columns.to_list()
        self.df_class = df_class
        self.houses = ['Ravenclaw', 'Slytherin', 'Griffondor', 'Hufflepuff']
        #self.houses = df_class['Hogwarts House'].unique()
        self.df_losses = pd.DataFrame(columns=self.houses)
        self.df_losses.fillna(0)
        self.df_weights = pd.DataFrame(columns=self.houses, index=self.features)
        self.df_weights.fillna(0)

    def train_one_vs_all(self, house, y_actual):
        loss = []
        weights = np.ones(len(self.features)).T
        for iter in range(self.epochs):
            z_output = np.dot(self.x_train, weights)
            h_pred = LogRegTrain.sigmoid(z_output) 
            loss_iter = LogRegTrain.loss_function(y_actual, h_pred)
            loss.append(loss_iter)
            gradient = np.dot(self.x_train.T, (h_pred - y_actual))
            grad_desc = LogRegTrain.gradient_descent(self.x_train, h_pred, gradient)
            weights = LogRegTrain.update_weight_loss(weights, self.learning_rate, grad_desc)
            return loss, weights

    def train(self, learning_rate=0.1, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        for house in self.houses:
            y_actual = np.where(self.df_class[1] == house, 1, 0)
            loss, weights = self.train_one_vs_all(house, y_actual)
            self.df_losses[house] = loss
            self.df_weights[house] = weights

    @staticmethod
    def standardize(arr:np.ndarray):
        mean = np.mean(arr)
        std = np.std(arr)
        return (arr - mean) / std
    
    @staticmethod
    def sigmoid(arr:np.ndarray):
        return 1 / (1 + np.exp(-arr))
    
    @staticmethod
    def loss_function(y_actual, h_pred):
        """ y_actual : target class. 1 in class, 0 not in class
        h_pred = signoid(x.weights)
        loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        """
        m = len(h_pred)
        a = -y_actual * np.log(h_pred)
        b = (1 - y_actual) * np.log(1 - h_pred)
        return (a - b) / m
    
    @staticmethod
    def gradient_descent(x_train, h_pred, y_actual):
        return np.dot(x_train.T, (h_pred - y_actual)) / y_actual.shape[0]

    @staticmethod
    def update_weight_loss(weights, learning_rate, grad_desc):
        return weights - learning_rate * grad_desc

In [30]:

df = pd.read_csv(f'../datasets/dataset_train.csv')
df_class = df['Hogwarts House']
df_features = df[df.columns[6:]]
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df_features = df_features.drop(excluded_features, axis=1)
# unstandardized data to train
logreg_model = LogRegTrain(df_features, df_class)
logreg_model.train(0.1, 50)
sns.lineplot(data=logreg_model.df_losses, legend='auto')

ValueError: operands could not be broadcast together with shapes (1600,) (10,) 